In [2]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os
import re


In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass
    
    def _get_url(self, param_dict: dict) -> str:
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = (
            'https://lenta.ru/search/v2/process?'
            + 'from={}&'.format(param_dict['from'])
            + 'size={}&'.format(param_dict['size'])
            + 'sort={}&'.format(param_dict['sort'])
            + 'title_only={}&'.format(param_dict['title_only'])
            + 'domain={}&'.format(param_dict['domain'])
            + 'modified%2Cformat=yyyy-MM-dd&'
            + 'type={}&'.format(param_dict['type']) * hasType
            + 'bloc={}&'.format(param_dict['bloc']) * hasBloc
            + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])
            + 'modified%2Cto={}&'.format(param_dict['dateTo'])
            + 'query={}'.format(param_dict['query'])
        )
        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json().get('matches', []))
        return search_table

    def get_articles(self, param_dict, time_step=10) -> pd.DataFrame:
        """
        Загружает статьи, объединяя их через pd.concat без сохранения на диск.
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')

        if dateFrom > dateTo:
            raise ValueError('dateFrom должен быть меньше dateTo')

        all_articles = pd.DataFrame()

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print(f"Обработка статей с {param_copy['dateFrom']} по {param_copy['dateTo']}")

            try:
                current_batch = self._get_search_table(param_copy)
                all_articles = pd.concat([all_articles, current_batch], ignore_index=True)
            except Exception as e:
                print(f"Ошибка при обработке периода {param_copy['dateFrom']} - {param_copy['dateTo']}: {e}")

            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')

        print('Обработка завершена.')
        return all_articles

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass
    
    def _get_url(self, param_dict: dict) -> str:
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = (
            'https://lenta.ru/search/v2/process?'
            + 'from={}&'.format(param_dict['from'])
            + 'size={}&'.format(param_dict['size'])
            + 'sort={}&'.format(param_dict['sort'])
            + 'title_only={}&'.format(param_dict['title_only'])
            + 'domain={}&'.format(param_dict['domain'])
            + 'modified%2Cformat=yyyy-MM-dd&'
            + 'type={}&'.format(param_dict['type']) * hasType
            + 'bloc={}&'.format(param_dict['bloc']) * hasBloc
            + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])
            + 'modified%2Cto={}&'.format(param_dict['dateTo'])
            + 'query={}'.format(param_dict['query'])
        )
        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json().get('matches', []))
        return search_table

    def get_articles(self, param_dict, time_step=10) -> pd.DataFrame:
        """
        Загружает статьи, объединяя их через pd.concat без сохранения на диск.
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')

        if dateFrom > dateTo:
            raise ValueError('dateFrom должен быть меньше dateTo')

        all_articles = pd.DataFrame()

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print(f"Обработка статей с {param_copy['dateFrom']} по {param_copy['dateTo']}")

            try:
                current_batch = self._get_search_table(param_copy)
                all_articles = pd.concat([all_articles, current_batch], ignore_index=True)
            except Exception as e:
                print(f"Ошибка при обработке периода {param_copy['dateFrom']} - {param_copy['dateTo']}: {e}")

            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')

        print('Обработка завершена.')
        return all_articles

In [ ]:
from IPython.display import clear_output
use_parser = "LentaRu"
query = ''
offset = 0
size = 850
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0"

# Проверка и инициализация парсера
assert use_parser == "LentaRu"
parser = lentaRu_parser()

# Списки временных периодов
years = ['2023','2024']
combined_data = pd.DataFrame()  # Общий DataFrame

for year in years:
    dateFrom = f"{year}-01-01"
    dateTo = f"{year}-12-31" if year != '2024' else '2024-12-23'  # Ограничение для 2024 года
    
    param_dict = {
        'query': query,
        'from': str(offset),
        'size': str(size),
        'dateFrom': dateFrom,
        'dateTo': dateTo,
        'sort': sort,
        'title_only': title_only,
        'type': material,
        'bloc': bloc,
        'domain': domain,
    }
    
    print(f"Скачиваем данные за {year}: {param_dict}")
    yearly_data = parser.get_articles(param_dict=param_dict, time_step=10)
    combined_data = pd.concat([combined_data, yearly_data], ignore_index=True)  # Объединяем данные

# Итоговый результат
print(f"Всего статей загружено: {len(combined_data)}")
combined_data.head()

In [ ]:
# Проверяем 
print(f"до удаления дубликатов было записей: {len(combined_data)}")

# Создаем новый столбец с первыми 30 символами из 'rightcol'
combined_data['rightcol_50'] = combined_data['rightcol'].apply(lambda x: str(x)[:30])

# Убираем дубликаты по новому столбцу
combined_data = combined_data.drop_duplicates(subset=['rightcol_50'], keep='first')



# Проверяем результат
print(f"После удаления дубликатов осталось записей: {len(combined_data)}")

In [ ]:
tag = combined_data.copy()


# Список ключевых слов, связанных со строительством
construction_keywords = [
    'строительство', 'проект', 'ремонт', 'застройка', 'строить',
     'площадка', 'архитектура', 'строительные', 'монтаж','стройка'
]

# Создаем регулярное выражение для поиска этих слов
construction_pattern = re.compile('|'.join(construction_keywords), re.IGNORECASE)

# Сначала применяем маппинг по TagsMap для замены значений в столбце 'topic'
TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5}
combined_data['topic'] = combined_data['bloc'].map(TagsMap)

# Теперь ищем ключевые слова для категории 6 (строительство) и заполняем столбец 'topic'
combined_data['topic'] = combined_data.apply(
    lambda row: 6 if bool(construction_pattern.search(str(row['text']))) else row['topic'],
    axis=1
)

# Проверяем, сколько записей попало в категорию 6
print(f"Количество записей в категории 6: {combined_data['topic'].value_counts().get(6, 0)}")

# Фильтруем только нужные значения по столбцу 'topic'
combined_data = combined_data.dropna(subset=['topic'])

In [ ]:
combined_data.to_csv(r'D:\hse_hw\news.csv',index=False)

In [3]:
tbl = pd.read_csv(r'D:\hse_hw\news.csv')

In [3]:
tbl['topic'].value_counts()

topic
0.0    7000
6.0    4917
3.0    4757
1.0    3343
2.0    3241
4.0    2412
8.0    1603
7.0    1374
5.0    1095
Name: count, dtype: int64

In [7]:
import pandas as pd
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
russian_stop_words = set(stopwords.words('russian'))

# Преобразование текста в строковый формат
tbl['text'] = tbl['text'].astype(str)

# Предварительное удаление русских стоп-слов
def remove_stop_words(text):
    words = text.lower().split()
    return ' '.join(word for word in words if word not in russian_stop_words)

tbl['cleaned_text'] = tbl['text'].apply(remove_stop_words)

# Использование TfidfVectorizer для анализа текста
vectorizer = TfidfVectorizer(max_features=10)
tfidf_matrix = vectorizer.fit_transform(tbl['cleaned_text'])
top_words = vectorizer.get_feature_names_out()
top_scores = tfidf_matrix.sum(axis=0).A1

# Создание DataFrame для топ слов
top_words_df = pd.DataFrame({"word": top_words, "score": top_scores})

# Гистограмма топ слов
fig_word_hist = px.bar(top_words_df, x='word', y='score',
                       title='Топ-10 слов по TF-IDF',
                       labels={'word': 'Слово', 'score': 'TF-IDF вес'},
                       template='plotly_white')

# Анализ длины сообщений
tbl['message_length'] = tbl['text'].apply(len)

# Гистограмма распределения длин сообщений
fig_length_hist = px.histogram(tbl, x='message_length', nbins=50,
                               title='Распределение длины новостей',
                               labels={'message_length': 'Длина новостей'},
                               template='plotly_white')

# Примеры данных
sample_messages = tbl['text'].head(2).tolist()

# Вывод результатов анализа и графиков
print("Примеры новостей:")
for idx, message in enumerate(sample_messages, 1):
    print(f"{idx}: {message}")


# Отображение графиков
fig_length_hist.show()
fig_word_hist.show()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\galiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Примеры новостей:
1: Фото: ЦТАК / Reuters Марина Совина Лидер КНДР Ким Чен Ын пообещал нарастить производство ядерных боеголовок «по экспоненте». Об этом пишет The Washington Post со ссылкой на северокорейские СМИ. Такое заявление прозвучало в ходе собрания правящей партии Кореи, спустя несколько часов после запуска нескольких баллистических ракет в сторону Японского моря. Выступая перед однопартийцами, Ким также сообщил  о планах построить более мощную межконтинентальную баллистическую ракету. Ранее Ким Чен Ын заявил, что ракеты из новых пусковых установок КНДР способны нести тактическое ядерное оружие, в пределах досягаемости которого находится вся Южная Корея. До этого Пхеньян пригрозил ответить ядерным оружием на ядерное оружие. Как уточнили военные в КНДР, проведенное в ноябре испытание новой межконтинентальной баллистической ракеты «Хвасон-17» доказало готовность Северной Кореи соответствующим образом отреагировать на ядерные угрозы.
2: Фото: Кирилл Шипицин / РИА Новости Марина С